# Agenda

1. Magic methods
2. Context managers (how objects can behave in `with` blocks)
3. Static and class methods
4. Multiple inheritance and the MRO 
5. Python object model + hierarchy
6. Properties (looks like data, acts like methods)
7. Descriptors
8. Dataclasses

# Magic methods

We can define methods on our classes, and then access them via our instances. Generally speaking, we only want a method to be invoked when we do it explicitly.

But there are a whole slew of methods, which we call "dunder methods," or "magic methods," which we almost never invoke directly ourselves.  Rather, Python looks for these methods, and then invokes them under certain circumstances.

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
p = Person('name1')    
print(p.greet())

Hello, name1!


In [2]:
vars(p)  # this returns the dict of attributes set on the instance

{'name': 'name1'}

In [3]:
p.name  # retrieve the attribute named "name" from the object that p refers to


'name1'

In [4]:
# ICPO -- attribute lookup: instance, class, parent, object

print(p)   # this calls p.__str__() 

In [7]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    
    def __repr__(self):
        return f'Instance of Person, {vars(self)=}'    # as of Python 3.8, = after variable in f-string
    
p = Person('name1')    
print(p.greet())
print(p)

Hello, name1!
Instance of Person, vars(self)={'name': 'name1'}


In [8]:
# example: len(something)

# when we call len(something), the "len" function calls something.__len__() 

len(p)  # what's the length of our person?

TypeError: object of type 'Person' has no len()

In [10]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Instance of Person, {vars(self)=}'    # as of Python 3.8, = after variable in f-string
    
    def __len__(self):
        return len(self.name)  # how long is the name?
    
p = Person('name1')    
print(p.greet())
print(p)
print(len(p))  # I call len(p)... len calls p.__len__() 

Hello, name1!
Instance of Person, vars(self)={'name': 'name1'}
5


In [12]:
p[3]   # what happens when I do this?

TypeError: 'Person' object is not subscriptable

In [13]:
# when we say x[i], what's really being run behind the scenes is x.__getitem__(i)
# [] are "syntactic sugar," easier for us to write and read, but rewritten behind the scenes by the language

In [18]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Instance of Person, {vars(self)=}'    # as of Python 3.8, = after variable in f-string
    
    def __len__(self):
        return len(self.name)  # how long is the name?
    
    def __getitem__(self, index):
        print(f'\t{index=}')
        return self.name[index]
    
p = Person('name1')    
print(p[3])    # this is like saying p.__getitem__(3), which will return p.name[3]

	index=3
e


In [19]:
print(p[-100])

	index=-100


IndexError: string index out of range

In [20]:
# what if I ask for a slice?
print(p[1:4])   # slice from index 1 until (and not including) index 4

	index=slice(1, 4, None)
ame


In [21]:
slice(5)   # same as saying [:5]

slice(None, 5, None)

In [22]:
slice(1,4)

slice(1, 4, None)

In [23]:
slice(1,4,2)  # same as [1:4:2]

slice(1, 4, 2)

In [24]:
s = 'abcdefghijklmnopqrstuvwxyz'

s[10:20]   # from 10 until (not including) 20

'klmnopqrst'

In [25]:
# the above is rewritten to be:
s[slice(10, 20)]     # [10:20] is turned into this

'klmnopqrst'

In [26]:
s[10:20:3]   # same as s[slice(10, 20, 3)]

'knqt'

# What happens when we add objects together with `+`?

```python
3 + 5      # adding two integers, we get a new integer
'a' + 'bc' # adding two strings, we get a new string
```

Whenever we add two objects in Python, we're actually invoking a method. The `+` operator is translated into a call to the `__add__` method.

If I say `x + y` in Python, the language rewrites this to be `x.__add__(y)`.  In other words, we invoke the `__add__` method on whatever is on the *left* side. The second operand is then passed as an argument.

In [27]:
p1 = Person('name1')
p2 = Person('name2')

p1 + p2   # what will happen now?

TypeError: unsupported operand type(s) for +: 'Person' and 'Person'

In [28]:
x = 10
y = '20'

x + y   

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [29]:
# what happens if I add them, but in the opposite order?

y + x

TypeError: can only concatenate str (not "int") to str

In [36]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __add__(self, other):          # other is the object on the right side of the +
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(self.name + other.name)  # create a new person with the combined name
        else:
            return Person(self.name + str(other))
    
p1 = Person('name1')
p2 = Person('name2')

p1 + p2  # this invokes p1.__add__(p2) --> Person.__add__(p1, p2)

Person object, vars(self)={'name': 'name1name2'}

In [37]:
p1 + 'xxxxx'

Person object, vars(self)={'name': 'name1xxxxx'}

In [38]:
'yyyyy' + p1      # 'yyyyy'.__add__(p1)

TypeError: can only concatenate str (not "Person") to str

In [39]:
# If we try to invoke a method like __add__ and the method doesn't know what to do
# with an "other" argument of the current type, it will raise an exception, as 
# we've seen  *BUT FIRST* it will try to invoke the method in the opposite direction.

# Python does this via a value that it returns from the method called
# NotImplemented

NotImplemented

NotImplemented

In [40]:
type(NotImplemented)

NotImplementedType

In [41]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __add__(self, other):          # other is the object on the right side of the +
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(self.name + other.name)  # create a new person with the combined name
        else:
            return Person(self.name + str(other))
        
    def __radd__(self, other):   # this is invoked when Python tries +, swapping the order of the arguments
        return self.__add__(other)
    
p1 = Person('name1')
p2 = Person('name2')

p1 + p2  # this invokes p1.__add__(p2) --> Person.__add__(p1, p2)

Person object, vars(self)={'name': 'name1name2'}

In [42]:
p2 + p1

Person object, vars(self)={'name': 'name2name1'}

In [43]:
p1 + 'xxxx'

Person object, vars(self)={'name': 'name1xxxx'}

In [44]:
'yyy' + p1

Person object, vars(self)={'name': 'name1yyy'}

In [45]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __add__(self, other):          # other is the object on the right side of the +
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(self.name + other.name)  # create a new person with the combined name
        else:
            return Person(self.name + str(other))
        
    def __radd__(self, other):   # this is invoked when Python tries +, swapping the order of the arguments
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(other.name + self.name)
        else:
            return Person(str(other) + self.name)

    
p1 = Person('name1')
p2 = Person('name2')
p3 = Person('a' + 'b' + 'c')


p1 + p2  # this invokes p1.__add__(p2) --> Person.__add__(p1, p2)

Person object, vars(self)={'name': 'name1name2'}

In [46]:
p1 + 'xxxxx'

Person object, vars(self)={'name': 'name1xxxxx'}

In [47]:
'yyyyy' + p2

Person object, vars(self)={'name': 'yyyyyname2'}

# Exercise: SortedList

1. Define a new class, `SortedList`, that we'll assume will only contain integers.
2. When you create a new instance of `SortedList`, you pass a list of integers.  These are then stored internally, in sorted order.  (It's totally OK to use `sorted` in this exercise.)
3. Have `__repr__` return a string with the sorted items.
4. Have `__len__` return the number of items in our list.
5. If you invoke `+` on a `SortedList` and another `SortedList`, the result is a new `SortedList` instance with all of the original elements, in sorted order.
6. If you invoke `+` on something else (should be an integer), then that should be added, in its appropriate place, to the `SortedList` instance.

Example:

    sl1 = SortedList([10, 5, 18])
    sl2 = SortedList([9,7,12])
    print(sl1)  # [5, 10, 18]
    print(sl2)  # [7, 9, 12]

    sl3 = sl1 + 7 
    print(sl3)  # [5, 7, 10, 18]
    
    sl4 = 8 + sl1
    print(sl4)  # [5, 8, 10, 18]
    
    print(sl1 + sl2)   # [5, 7, 9, 10, 12, 18]

In [68]:
class SortedList:
    def __init__(self, numbers):
        self.numbers = sorted(numbers)
        
    def __repr__(self):
        return str(self.numbers)
    
    def __add__(self, other):
        if hasattr(other, 'numbers'):
            return SortedList(self.numbers + other.numbers)
        else:
            return SortedList(self.numbers + [int(other)])
        
    def __radd__(self, other):
        return self.__add__(other)
        
sl1 = SortedList([10, 5, 18])
print(sl1)

sl2 = SortedList([9, 7, 12])
print(sl2)

[5, 10, 18]
[7, 9, 12]


In [69]:
print(sl1 + sl2)

[5, 7, 9, 10, 12, 18]


In [70]:
sl3 = sl1 + 7
print(sl3)

[5, 7, 10, 18]


In [71]:
sl4 = 8 + sl1
print(sl4)

[5, 8, 10, 18]


In [72]:
mylist = [10, 20, 30]
mylist += [40, 50, 60]    

# result is that mylist has changed!
mylist

[10, 20, 30, 40, 50, 60]

In [73]:
sl1 

[5, 10, 18]

In [74]:
sl2

[7, 9, 12]

In [75]:
sl1 += sl2   # what will this do?  will it even work?  yes, it's just like saying sl1 = sl1 + sl2

In [76]:
sl1

[5, 7, 9, 10, 12, 18]

In [ ]:
# when you use +=, Python turns that into +, and then turns it into assignment (=)
# but it doesn't have to!

In [77]:
class SortedList:
    def __init__(self, numbers):
        self.numbers = sorted(numbers)
        
    def __repr__(self):
        return str(self.numbers)
    
    def __add__(self, other):
        if hasattr(other, 'numbers'):
            return SortedList(self.numbers + other.numbers)
        else:
            return SortedList(self.numbers + [int(other)])
        
    def __radd__(self, other):
        return self.__add__(other)
    
    def __iadd__(self, other):  # this is for += "inplace add"
        print(f'\tNow in __iadd__!')
        if hasattr(other, 'numbers'):
            self.numbers = sorted(self.numbers + other.numbers)
        else:
            self.numbers = sorted(self.numbers + [int(other)])
            
        return self
        
sl1 = SortedList([10, 5, 18])
print(sl1)

sl2 = SortedList([9, 7, 12])
print(sl2)

[5, 10, 18]
[7, 9, 12]


In [78]:
sl1

[5, 10, 18]

In [79]:
sl2

[7, 9, 12]

In [80]:
sl1 += sl2

	Now in __iadd__!


In [81]:
sl1

[5, 7, 9, 10, 12, 18]

In [82]:
2 + 3

5

In [83]:
2 += 3

SyntaxError: 'literal' is an illegal expression for augmented assignment (<ipython-input-83-00e9f52d1826>, line 1)

# Next up

- Equality and comparisons
- Slots
- Context managers
- Multiple inheritance

In [84]:
# Return at :15

In [85]:
t = ([10, 20, 30], 
    [100, 200, 300])

len(t)

2

In [86]:
t[0].append(40)
t

([10, 20, 30, 40], [100, 200, 300])

In [87]:
t[0] += [50, 60, 70]

TypeError: 'tuple' object does not support item assignment

In [88]:
t

([10, 20, 30, 40, 50, 60, 70], [100, 200, 300])

In [102]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __eq__(self, other):
        if hasattr(other, 'name'):
            return self.name == other.name
        
        return False
    
    def __hash__(self):
        return hash(self.name)

p1 = Person('Adam')
p2 = Person('Adam')

In [103]:
p1 == p2

True

In [104]:
d = {'a':1}   # 'a' is the key, and 1 is the value
d

{'a': 1}

In [105]:
# keys have to be hashable -- not exactly immutable, and there are mutable, hashable objects
# for example: any instance of our classes!

# however, if we define __eq__, then we must also define __hash__ in order for our objects to be hashable

d[p1] = 10

In [106]:
d[p1]

10

In [107]:
d[p2]

10

# How can our objects be hashable?

- By default, our objects are hashable. You don't need to do anything.
- If you define `__eq__`, then the object is no longer hashable, unless you also define `__hash__`.
- If you do *not* define `__eq__`, then you can still define `__hash__`, to make the object hashable in a custom way.

In [111]:
class Foo:
    def __init__(self, x):
        self.x = x
        
    def __hash__(self):
        return 10
        
f = Foo(10)        
d[f] = 1

In [114]:
d[f]

1

In [113]:
f2 = Foo(20)
d[f2]

KeyError: <__main__.Foo object at 0x110506670>

# Context managers

You've probably seen (and used) many times, the following sort of code:

```python
with open('myfile.txt') as f:
    for one_line in f:
        print(len(one_line), end=' ')
    # here, the file is automatically flushed + closed
```

Here's what's really going on behind the scenes:

```python
with open('myfile.txt') as f:

    # Here, Python automatically calls f.__enter__()

    for one_line in f:
        print(len(one_line), end=' ')

    # here, Python automatically calls f.__exit__()
```

Any object that implements `__enter__` and `__exit__` is called a "context manager." Files are, far and away, the most common uses for context managers. But there are others out there -- locks, database transactions, etc.

In [124]:
class MyContext:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
        
    def __enter__(self):
        print(f'Now in __enter__, {vars(self)=}!')
        return self   # this returned value is assigned to c, in "as c" in the with block
    
    def __exit__(self, *args):
        print(f'Now in __exit__, {vars(self)=}!')
        if any(args):
            print(f'You got an exception: {args}')
        return True
    
with MyContext('abcde') as c:
    print('Hello!')
    print(c.x2())
    print('Goodbye!')

Now in __enter__, vars(self)={'x': 'abcde'}!
Hello!
abcdeabcde
Goodbye!
Now in __exit__, vars(self)={'x': 'abcde'}!


# Exercise: TempStdout

1. Normally, when we `print`, output goes to the file object known as `sys.stdout`.
2. (Yes, it's possible in modern Python to pass `file` as an argument to `print`, and have it print elsewhere.)
3. Create a context manager, `TempStdout`, which takes an argument, a writable file, which will be assigned to `sys.stdout` for the duration of the `with` block. 
4. When the `with` block exits, we restore `stdout` to what it was before (presumably, the screen).

Example:

```python
print('Before')  # screen

with TempStdout(open('mystdout.txt', 'w')) as s:
    print('During')   # printed to mystdout.txt
    
print('After')  # screen
```

In [130]:
import sys

class TempStdout:
    def __init__(self, f):
        self.f = f
        self.oldstdout = None
        
    def __enter__(self):
        print(f'Now in __enter__')
        sys.oldstdout = sys.stdout   # save the existing sys.stdout somewhere!
        sys.stdout = self.f
        return self
        
    def __exit__(self, *args):
        sys.stdout = sys.oldstdout
        self.f.close()  # also flushes it
        self.oldstdout = None
        print(f'Now in __exit__')
        return True

print('Before')  # screen

with TempStdout(open('mystdout.txt', 'w')) as s:
    print('During')   # printed to mystdout.txt
    
print('After')  # screen

Before
Now in __enter__
Now in __exit__
After


In [129]:
!cat mystdout.txt

During


In [131]:
class Foo:
    def __init__(self, x):
        self.x = x
        
f = Foo(10)        
print(vars(f))

{'x': 10}


In [132]:
f.x = 20
print(vars(f))

{'x': 20}


In [133]:
# what if I now add a new attribute?
f.y = 'hahaahahaha I added a new attribute'
print(vars(f))

{'x': 20, 'y': 'hahaahahaha I added a new attribute'}


In [135]:
class Foo:
    # class attribute: __slots__
    __slots__ = ['x']  

    def __init__(self, x):
        self.x = x
        
f = Foo(10)        

In [136]:
f.x

10

In [137]:
vars(f)

TypeError: vars() argument must have __dict__ attribute

In [138]:
f.y = 20

AttributeError: 'Foo' object has no attribute 'y'

In [140]:
class Foo:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
    
    def hello(self):
        return 'Hello from Foo!'
    
f = Foo(10)    
f.x2()

20

In [141]:
f.hello()

'Hello from Foo!'

In [142]:
# what if I want to call my methods on my class, Foo?  Can I do that?

Foo.x2(f)  # I have to provide an instance of Foo as the argument

20

In [143]:
Foo.hello(f)

'Hello from Foo!'

In [144]:
Foo.hello()

TypeError: hello() missing 1 required positional argument: 'self'

In [145]:
class Foo:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
    
    def hello():
        return 'Hello from Foo!'
    
f = Foo(10)    
f.x2()

20

In [146]:
Foo.hello()

'Hello from Foo!'

In [148]:
f.hello()   # rewrote this to be Foo.hello(f)

TypeError: hello() takes 0 positional arguments but 1 was given

# Static methods 

Static methods are methods that we can call via either the class or the instance, but which don't expect/require/get `self` as the first parameter. They can take any other parameters we want, but they don't have access to `self`, and thus don't have access to the state of our instance.

In [149]:
class Foo:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
    
    @staticmethod     # decorator before the method definition
    def hello():
        return 'Hello from Foo!'
    
f = Foo(10)    
f.x2()

20

In [150]:
Foo.hello()

'Hello from Foo!'

In [151]:
f.hello()

'Hello from Foo!'

# What are class methods, then?

In Python, a class method is a method that can be called from either the class or the instance. Like an instance method, it gets an argument passed to it automatically, in the first position. But unlike an instance method, that first argument is a *class*, the class on which we're running our method.

So `self` is the first parameter in every instance method, and is the instance on which we're running the method.

And `cls` (the traditional name given to the first parameter in a class method) is the first parameter in every class method, and is the *class* on which we're running things (aka `type(self)`).

In [152]:
class Foo:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
    
    @staticmethod     # decorator before the method definition
    def hello():
        return 'Hello from Foo!'
    
    @classmethod      # decorator before the method definition
    def goodbye(cls):
        return f'Goodbye from Foo, class = {cls}'
    
f = Foo(10)    
f.x2()

20

In [153]:
f.goodbye()

"Goodbye from Foo, class = <class '__main__.Foo'>"

In [154]:
Foo.goodbye()

"Goodbye from Foo, class = <class '__main__.Foo'>"

# Next up:

- Multiple inheritance and the MRO
- Python object model
- Metaclasses

In [155]:
# Return :06

# ICPO rule

If we ask for an attribute on an object, then this is Python's search path for that attribute:

- `I` instance
- `C` instance's class
- `P` parent of the instance's class
- `O` `object`, the top of the object hierarchy in Python

In [158]:
class A:
    def __init__(self, x):
        self.x = x
        
    def hello(self):
        return f'Hello from A, where {self.x=}'
        
class B(A):   # B is a subclass of A, aka it inherits from A
    def __init__(self, x, y):
        super().__init__(x)   # this runs A.__init__(x), thus adding x as an attribute on our instance
        self.y = y            # this adds y as an attribute on our instance
        
    def goodbye(self):
        return f'Goodbye from B, where {self.y=}'
        
b = B(10, 20)

In [159]:
b.goodbye()  # instance - no, class B - yes, it has an attribute named goodbye

'Goodbye from B, where self.y=20'

In [160]:
b.hello()    # instance - no, class B - no, is it on B's parent? YES -- A has an attribute named hello

'Hello from A, where self.x=10'

In [161]:
b.__repr__()  # instance - no, class B - no, class A - no, object 

'<__main__.B object at 0x11057dd60>'

In [162]:
# Never does a method sit on an instance.
# 100% of methods in Python are class attributes.
# They are defined in the "class" definition.

In [163]:
# In order for methods to work in Python, we always do this lookup:
# - first look on the instance for the method. It isn't there.
# - then look on the class for the method. It'll usually be there.
# - if not, then look on the parent

In [165]:
b.y  # methods aren't on the instances, but data attributes *are*

20

In [166]:
# we can make it even worse:
# NEVER EVER EVER DO THIS IN REAL LIFE
# (unless your officemate goes to make a cup of coffee)

b.goodbye = lambda : 'Ha ha ha I hijacked goodbye'


In [167]:
b.goodbye()  # searches on the instance

'Ha ha ha I hijacked goodbye'

In [168]:
del(b.goodbye)  # remove this instance attribute

In [169]:
b.goodbye()

'Goodbye from B, where self.y=20'

In [170]:
class A:
    def __init__(self, x):
        self.x = x
        
    def hello(self):
        return f'Hello from A, where {self.x=}'
        
class N:
    def __init__(self, z):
        self.z = z
        
    def whatever(self):
        return f'Hello from N, where {self.z=}'

# multiple inheritance
class B(A, N):   # B is a subclass of both A and N
    def __init__(self, x, y, z):
        A.__init__(self, x)  # initialize attributes for A
        N.__init__(self, z)  # initialize attributes for N

        self.y = y            # this adds y as an attribute on our instance
        
    def goodbye(self):
        return f'Goodbye from B, where {self.y=}'
        
# what if we want B to inherit from not just A, but also something else?

b = B(10, 20, 30)

In [171]:
b.goodbye()  # on b? No. On B? Yes.

'Goodbye from B, where self.y=20'

In [173]:
b.hello()  # on b? No. On B? No. On A, the first parent? Yes.

'Hello from A, where self.x=10'

In [174]:
b.whatever()  # on b? No. On B? No. On A, the first parent? No. On N, the second parent? YES.

'Hello from N, where self.z=30'

In [175]:
# instance is b
# class is B  (capital B)
# parents of B are both A and N


In [176]:
# how can we know what a class inherits from ? Check its __bases__ attribute, a tuple
B.__bases__

(__main__.A, __main__.N)

In [177]:
A.__bases__

(object,)

In [178]:
N.__bases__

(object,)

In [179]:
object.__bases__

()

In [180]:
# summarize this as MRO -- method resolution order
B.__mro__

(__main__.B, __main__.A, __main__.N, object)

In [181]:
# if you use multiple inheritance, then the classes in __bases__ must always have
# children before parents.

# if you try to put a parent class before a child class in the __bases__ attribute,
# then Python will refuse to define the class.

In [182]:
b.__mro__ # what is the __mro__ of the instance?

AttributeError: 'B' object has no attribute '__mro__'

In [184]:
B.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.B.__init__(self, x, y, z)>,
              'goodbye': <function __main__.B.goodbye(self)>,
              '__doc__': None})

In [186]:
class MyClass(B, A, N):   # Python allows this, because B is a child of A, A is a child of N
    pass

In [187]:
class MyClass(N, A, B):  # Python will *not* allow this, because B is a child of A, but comes after it
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases N, A, B

# Mixins

A mixin is class that you inherit from, which has two qualities:

1. It doesn't define any data attributes of its own
2. It defines a small number of methods (often just one) that are designed to override methods in later classes in the MRO.

In other words, instead of inheriting from one class, you inherit from two classes.  You inherit from the first (the mixin) so that it'll shadow, or block, your access to a method in the second class.

In [194]:
class GenericPerson:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person instance, {vars(self)=}'
    
class FancyReprMixin:
    def __repr__(self):
        return f'*** repr = {vars(self)} ***'
    
class Person(FancyReprMixin, GenericPerson):   # add the mixin class *BEFORE* the real parent
    pass

p = Person('name1')
print(p.greet())   # p has greet? no. p's class Person has greet? no. FancyReprMixin has greet? No. Genericperson has greet? yes
print(p)  # p has __repr__? no. p's class Person has __repr__? no. FancyReprMixin has __repr__? Yes.

Hello, name1!
*** repr = {'name': 'name1'} ***


In [195]:
class SpecialType(type):
    pass

class UseSpecialType(metaclass=SpecialType):  # this is how you set a different metaclass
    pass

In [196]:
type(UseSpecialType)

__main__.SpecialType

In [197]:
class SpecialType(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.x = 100

class UseSpecialType(metaclass=SpecialType):  # this is how you set a different metaclass
    pass


# when I create a new object (of any sort)
# - we call the class  
# - that calls __new__
# - that calls __init__ on the class
# by the time __init__ is called, the new object exists, but it hasn't been returned to the caller

In [198]:
UseSpecialType.x   # class attribute set by my metaclass, SpecialType

100

# Next up:

- Properties
- Descriptors 
- Functions vs. methods

# Properties



In [199]:
# let's say that our company makes thermostats
# we allow people to control the thermostats in Python (of course)

class Thermostat:
    def __init__(self, temp):     # degrees Celsius
        self.temp = temp
        
t = Thermostat(20)
print(t.temp)
t.temp = 23
print(t.temp)

20
23


In [207]:
# how can we stop people from setting the temp to be too low, or too high?
# answer: we'll use setters and getters!

# but wait -- we have existing customers
# cannot break the existing API

# this is where *properties* come in
# a property is a special kind of attribute that looks like data, but works like a method

import time

class TempTooLowError(Exception):
    pass

class TempTooHighError(Exception):
    pass

class Thermostat:
    def __init__(self, temp):     # degrees Celsius
        self._temp = temp         # setting _temp, a "private" attribute on self
        
    @property                     # property decorator
    def temp(self):               # defining a getter method
        print(f'Calling temp getter method at {time.time()}')
        return self._temp         # return self._temp
    
    @temp.setter
    def temp(self, new_temp):
        print(f'Calling temp setter method with {new_temp} at {time.time()}')

        if new_temp < 8:
            raise TempTooLowError(f'Min temp is 8')
            
        if new_temp > 30:
            raise TempTooHighError('Max temp is 30')
            
        self._temp = new_temp
        
t = Thermostat(20)
print(t.temp)           # we're really calling the "temp" getter method

t.temp = 23
print(t.temp)

Calling temp getter method at 1632075508.640758
20
Calling temp setter method with 23 at 1632075508.640966
Calling temp getter method at 1632075508.641002
23


In [208]:
t.temp = 0

Calling temp setter method with 0 at 1632075610.580023


TempTooLowError: Min temp is 8

In [209]:
t.temp = 100

Calling temp setter method with 100 at 1632075620.682146


TempTooHighError: Max temp is 30

# Properties

- Properties look like regular attributes, but they're actually calling methods behind the scenes
    - Retrieving from a property calls the getter method (under `@property`)
    - Setting a property calls the setter method (under `@temp.setter`)
- In your getter/setter methods, you can do whatever you want
- I'd argue that you usually want to use properties whenever you have an API change
- This allows you to start with very simple object definitions and attributes, and over time enforce all sorts of value limitations/guard rails

Properties only work because they are class attributes accessed via the instance.  

In [210]:
# how can we stop people from setting the temp to be too low, or too high?
# answer: we'll use setters and getters!

# but wait -- we have existing customers
# cannot break the existing API

# this is where *properties* come in
# a property is a special kind of attribute that looks like data, but works like a method

import time

class TempTooLowError(Exception):
    pass

class TempTooHighError(Exception):
    pass

class Thermostat:
    def __init__(self):     # degrees Celsius
        self._temp = None         # setting _temp, a "private" attribute on self
        
    @property                     # property decorator
    def temp(self):               # defining a getter method
        print(f'Calling temp getter method at {time.time()}')
        return self._temp         # return self._temp
    
    @temp.setter
    def temp(self, new_temp):
        print(f'Calling temp setter method with {new_temp} at {time.time()}')

        if new_temp < 8:
            raise TempTooLowError(f'Min temp is 8')
            
        if new_temp > 30:
            raise TempTooHighError('Max temp is 30')
            
        self._temp = new_temp
        
t = Thermostat()
t.temp = 20
print(t.temp)           # we're really calling the "temp" getter method

t.temp = 23
print(t.temp)

Calling temp setter method with 20 at 1632075884.435084
Calling temp getter method at 1632075884.435403
20
Calling temp setter method with 23 at 1632075884.435454
Calling temp getter method at 1632075884.435683
23


In [214]:

import time

class TempTooLowError(Exception):
    pass

class TempTooHighError(Exception):
    pass

class Thermostat:
    def __init__(self, temp):     # degrees Celsius
        self.temp = temp          # setting temp, our property via the instance
        
    @property                     # property decorator
    def temp(self):               # defining a getter method
        print(f'Calling temp getter method at {time.time()}')
        return self._temp         # return self._temp
    
    @temp.setter
    def temp(self, new_temp):
        print(f'Calling temp setter method with {new_temp} at {time.time()}')

        if new_temp < 8:
            raise TempTooLowError(f'Min temp is 8')
            
        if new_temp > 30:
            raise TempTooHighError('Max temp is 30')
            
        self._temp = new_temp
        
t = Thermostat(20)
print(t.temp)           # we're really calling the "temp" getter method

t.temp = 23
print(t.temp)

Calling temp setter method with 20 at 1632076057.154473
Calling temp getter method at 1632076057.154729
20
Calling temp setter method with 23 at 1632076057.154944
Calling temp getter method at 1632076057.155029
23


In [213]:
Thermostat.temp

# Exercise: Randhist

1. Create a class, `Randhist`, that when you create an instance, and retrieve from the `rand` attribute, you'll get a random number.
2. When you retrieve from its `hist` attribute, you'll get a list of the random numbers that were previously returned by `rand`.
3. Assigning to `rand` (any value at all) will reset the history to be an empty list.
4. Remember that you can get a random number from 0-100 with `random.randint(0, 100)`.

Example:

    rh = RandHist()
    print(rh.rand)   # 35
    print(rh.rand)   # 82
    print(rh.rand)   # 6
    print(rh.hist)   # [35, 82, 6]
    rh.rand = None
    print(rh.hist)   # [ ]

In [223]:
import random

random.seed(0)

class RandHist:
    def __init__(self):
        self.hist = []
        
    @property
    def rand(self):
        n = random.randint(0, 100)
        self.hist.append(n)
        return n
    
    @rand.setter
    def rand(self, new_value):
        self.hist = []
    
rh = RandHist()
for i in range(5):
    print(rh.rand)

49
97
53
5
33


In [224]:
rh.hist

[49, 97, 53, 5, 33]

In [225]:
rh.rand = 'something'

In [226]:
rh.hist

[]

In [227]:
rh.rand

65

In [228]:
rh.hist

[65]

In [229]:
s = 'abcd'
s.upper()

'ABCD'

In [230]:
# the call to s.upper() is turned into
str.upper(s)

'ABCD'

# Descriptors

- How are method calls rewritten, so that we can either invoke them via the instance or the class?
- Who is rewriting things, so that our instance becomes `self`?
- When we retrieve a property, who is invoking the method?

The answer to all of these is: Descriptors.

Descriptors are the deepest part of Python objects.  They are:

- Objects defined as class attributes
- But you must retrieve the object via an instance
- When you retrieve the class attribute via the instance, the `__get__` method on that object is invoked
- When you set the class attribute via the instance, the `__set__` method on that object is invoked.

In other words, if we define a class that has `__get__` and `__set__` defined, and then create an instance of that class as a class attribute, and then retrieve that instance via an instance, then we'll use descriptors.

Another way to think about it: What if we have 100 different products in our company, all of which need temperature protection?  Instead of defining the property, one class at a time, we can define a new class called SafeTemp, and then use that on all of our products.

In [233]:
class SafeTemp:   # each instance of SafeTemp will be a class attribute on a product (e.g., Thermostat)

    def __init__(self):
        print('Now in SafeTemp.__init__\n')
        self._temp = None
        
    def __get__(self, instance, owner):
        print(f'Now in __get__, with {self=}, {instance=}, {owner=}\n')
        return self._temp
    
    def __set__(self, instance, new_temp):
        print(f'Now in __set__, with {self=}, {instance=}, {new_temp=}\n')
        self._temp = new_temp
        

print('Creating the Thermostat class')
class Thermostat:
    def __init__(self, temp):
        self.temp = temp   
        
    temp = SafeTemp()   # the "temp" class attribute on Thermostat is an instance of SafeTemp
    
print('Now creating the Thermostat instance')
t = Thermostat(20)
print(t.temp)        # retrieves the temp class attribute via the t instance
t.temp = 23          # setting the temp class attribute via the t instance
print(t.temp)        # retrieves the temp class attribute via the t instance

Creating the Thermostat class
Now in SafeTemp.__init__

Now creating the Thermostat instance
Now in __set__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591250>, new_temp=20

Now in __get__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591250>, owner=<class '__main__.Thermostat'>

20
Now in __set__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591250>, new_temp=23

Now in __get__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591250>, owner=<class '__main__.Thermostat'>

23


In [234]:
t2 = Thermostat(27)
t2.temp

Now in __set__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591160>, new_temp=27

Now in __get__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591160>, owner=<class '__main__.Thermostat'>



27

In [235]:
t.temp

Now in __get__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591250>, owner=<class '__main__.Thermostat'>



27

In [236]:
t2.temp = 20
t2.temp

Now in __set__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591160>, new_temp=20

Now in __get__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591160>, owner=<class '__main__.Thermostat'>



20

In [237]:
t.temp

Now in __get__, with self=<__main__.SafeTemp object at 0x110547d30>, instance=<__main__.Thermostat object at 0x110591250>, owner=<class '__main__.Thermostat'>



20

# The problem

We have `Thermostat`, a class.  It has a class attribute, `temp`.

If we retrieve from that attribute via an instance of `Thermostat`, then we're getting the value of the `SafeTemp` class attribute `_temp`.  

If we set the attribute `temp` on an instance of `Thermostat`, then we're setting the value of the `SafeTemp` class attribute `_temp`.

This is a problem if we have more than one thermostat.  What we need to do is distinguish between the various instances of `Thermostat`.

Fortunately, the second argument passed to `__get__` and `__set__` is the instance of `Thermostat` that was used to invoke the method.  Moreover, instances are hashable, and are thus usable as dict keys.

So... we'll use a dictionary to keep track of the values of temp for each instance of `Thermostat`.

In [241]:

class TempTooLowError(Exception):
    pass

class TempTooHighError(Exception):
    pass


class SafeTemp:   # each instance of SafeTemp will be a class attribute on a product (e.g., Thermostat)

    def __init__(self):
        print('Now in SafeTemp.__init__\n')
        self._temp = {}
        
    def __get__(self, instance, owner):
        print(f'Now in __get__, with {self=}, {instance=}, {owner=}\n')
        return self._temp[instance]
    
    def __set__(self, instance, new_temp):
        print(f'Now in __set__, with {self=}, {instance=}, {new_temp=}\n')

        if new_temp < 8:
            raise TempTooLowError(f'Min temp is 8')
            
        if new_temp > 30:
            raise TempTooHighError('Max temp is 30')

        self._temp[instance] = new_temp


print('Creating the Thermostat class')
class Thermostat:
    def __init__(self, temp):
        self.temp = temp   
        
    temp = SafeTemp()   # the "temp" class attribute on Thermostat is an instance of SafeTemp
    
print('Now creating the Thermostat instance')
t1 = Thermostat(20)
t2 = Thermostat(18)


Creating the Thermostat class
Now in SafeTemp.__init__

Now creating the Thermostat instance
Now in __set__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591dc0>, new_temp=20

Now in __set__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591610>, new_temp=18



In [242]:
print(t1.temp)

Now in __get__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591dc0>, owner=<class '__main__.Thermostat'>

20


In [243]:
print(t2.temp)

Now in __get__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591610>, owner=<class '__main__.Thermostat'>

18


In [244]:
t1.temp = 25
t2.temp = 24

Now in __set__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591dc0>, new_temp=25

Now in __set__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591610>, new_temp=24



In [245]:
print(t1.temp)

Now in __get__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591dc0>, owner=<class '__main__.Thermostat'>

25


In [246]:
print(t2.temp)

Now in __get__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591610>, owner=<class '__main__.Thermostat'>

24


In [247]:
t1.temp = 100

Now in __set__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591dc0>, new_temp=100



TempTooHighError: Max temp is 30

In [248]:
t2.temp = -5

Now in __set__, with self=<__main__.SafeTemp object at 0x11057d850>, instance=<__main__.Thermostat object at 0x110591610>, new_temp=-5



TempTooLowError: Min temp is 8